In [1]:
### Initialize

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interactive, interact, FloatSlider, IntRangeSlider
from dask.distributed import Client, progress, wait
import attengineshell as ate
import numpy as np
import dask.array as da
import dask
import sys
import os

sys.path.insert(0, '../thirdparty/singleTrace_d2geo/')
import CompleTrace as cmpAttr
from semblance import semblance

general_path = '/glb/data/cdis_projects/users/ussnis/attEngine'
inp_vt_file = general_path + '/data/fourpoint.vt'
filepath = os.path.basename(inp_vt_file)
name = os.path.splitext(filepath)[0]


In [2]:
### convert .vt to .zarr

out_zarr_file = '{0}/data_zarr'.format(general_path) 
print('out_zarr_file=' + out_zarr_file)

ate.FileConverter.vt_2_zarr(inp_vt_file, out_zarr_file, 300, 300, 300, False)

out_zarr_file=/glb/data/cdis_projects/users/ussnis/attEngine/data_zarr
load data to numpy array.
vt header filename is:/glb/data/cdis_projects/users/ussnis/attEngine/data_zarr/fourpoint/fourpoint.vt


In [2]:
### compute attributes

client = Client(address='tcp://10.53.2.146:8788', set_as_default=True, name='Client_2', direct_to_workers=True)
outputDir =  '{0}/dataOut/{1}'.format(general_path, name)
print('outputDir:'+ outputDir)
if os.path.exists(outputDir) == False:
    os.mkdir(outputDir)
    
inp_file_zarr = '{0}/data_zarr/{1}/{1}.zarr'.format(general_path, name)
print('inp_file_zarr:' + inp_file_zarr)

semblance_out_file_zarr = '{0}/dataOut/{1}/{1}_semblance.zarr'.format(general_path, name)
print('semblance_out_file_zarr:' + semblance_out_file_zarr)
semblance = ate.Semblance()
semblance.set_params(client, inp_file_zarr, semblance_out_file_zarr, False) 
data_in, res_semblance  = semblance.run()

# compute SingleTrace Envelope
envelope = ate.SingleTrace.Envelope()
singletrace_outfile = '{0}/dataOut/{1}/{1}_SingleTrace_Envelope.zarr'.format(general_path, name)
print('singletrace_outfile:' + singletrace_outfile)
envelope.set_params(client, data_in, singletrace_outfile, False)
data_in, res_envel = envelope.run_new()

# compute SingleTrace InstaneousPhase
instaphase = ate.SingleTrace.InstantaneousPhase()
singe_instananeous_outfile = '{0}/dataOut/{1}/{1}_SingleTrace_Instantaneous.zarr'.format(general_path, name)
print('singe_instananeous_outfile' + singe_instananeous_outfile)
instaphase.set_params(client, data_in, singe_instananeous_outfile, False)
data_in, res_instaphase = instaphase.run_new()


outputDir:/glb/data/cdis_projects/users/ussnis/attEngine/dataOut/fourpoint
inp_file_zarr:/glb/data/cdis_projects/users/ussnis/attEngine/data_zarr/fourpoint/fourpoint.zarr
semblance_out_file_zarr:/glb/data/cdis_projects/users/ussnis/attEngine/dataOut/fourpoint/fourpoint_semblance.zarr
/glb/data/cdis_projects/users/ussnis/attEngine/dataOut/fourpoint/fourpoint_semblance.zarr
singletrace_outfile:/glb/data/cdis_projects/users/ussnis/attEngine/dataOut/fourpoint/fourpoint_SingleTrace_Envelope.zarr
/glb/data/cdis_projects/users/ussnis/attEngine/dataOut/fourpoint/fourpoint_SingleTrace_Envelope.zarr
singe_instananeous_outfile/glb/data/cdis_projects/users/ussnis/attEngine/dataOut/fourpoint/fourpoint_SingleTrace_Instantaneous.zarr
/glb/data/cdis_projects/users/ussnis/attEngine/dataOut/fourpoint/fourpoint_SingleTrace_Instantaneous.zarr


In [5]:
plt.rcParams['figure.figsize'] = [16, 12]
def f(slice, xran, zran):
    xini = xran[0]
    xfin = xran[1]
    zini = zran[0]
    zfin = zran[1]

    if xfin <= xini or zfin <= zini:
        return
    plt.subplot(411)
    ate.plot_result_v01(np.rot90(data_in[xini:xfin,slice,zini:zfin],1))
    plt.title('Input data')
    plt.subplot(412)
    ate.plot_result_v01(np.rot90(res_semblance[xini:xfin,slice,zini:zfin],1))
    plt.title('Semblance')
    plt.subplot(413)
    ate.plot_result_v01(np.rot90(res_envel[xini:xfin,slice,zini:zfin],1))
    plt.title('Envelope')
    plt.subplot(414)
    ate.plot_result_v01(np.rot90(res_instaphase[xini:xfin,slice,zini:zfin],1))
    plt.title('Input data')
    
xmax, slicemax, zmax = data_in.shape

interact(f,slice=FloatSlider(value=940, min=0, max=slicemax-1, step=1, continuous_update=False),
         xran=IntRangeSlider(value=[0, 600], min=0, max=xmax-1, step=1, continuous_update=False),
         zran=IntRangeSlider(value=[0, 1000], min=0, max=zmax-1, step=1, continuous_update=False)
        )

interactive(children=(FloatSlider(value=400.0, continuous_update=False, description='slice', max=400.0, step=1…

<function __main__.f(slice, xran, zran)>

In [ ]:
### convert .zarr file to .vt
semblance_vt_file = '{0}/dataOut/{1}/{1}_semblance.vt'.format(general_path, name)
origin_vt_fmt_file = '{0}/data_zarr/{1}/{1}.vt'.format(general_path, name)
print('semblance_vt_file:' + semblance_vt_file)
if os.path.exists(semblance_vt_file):
    os.remove(semblance_vt_file)
    
print('Input file is:' + semblance_out_file_zarr)

ate.FileConverter.zarr_2_vt(zarr_file=semblance_out_file_zarr, vt_file=semblance_vt_file, original_path=origin_vt_fmt_file)

In [ ]:
### convert .vt to .numpy file to show

semblance_numpy_file = '{0}/dataOut/{1}/{1}_semblance.npy'.format(general_path, name)
print('semblance_numpy_file:' + semblance_numpy_file)
#ate.FileConverter.vt_2_numpyfile(semblance_vt_file, semblance_numpy_file)
#print('semblance_vt_file:' + semblance_vt_file)
#ate.FileConverter.zarr_2_numpyfile(semblance_out_file_zarr, semblance_numpy_file)
x=np.load(semblance_numpy_file)
a = da.from_array(x)
plt.rcParams['figure.figsize'] = [16, 12]
def f(slice, xran, zran):
    xini = xran[0]
    xfin = xran[1]
    zini = zran[0]
    zfin = zran[1]
       
    plt.subplot(412)
    ate.plot_result_v01(np.rot90(a[xini:xfin,slice,zini:zfin],1))
    
xmax, slicemax, zmax = a.shape
interact(f,slice=FloatSlider(value=940, min=0, max=slicemax-1, step=1, continuous_update=False),
         xran=IntRangeSlider(value=[0, 600], min=0, max=xmax-1, step=1, continuous_update=False),
         zran=IntRangeSlider(value=[0, 1000], min=0, max=zmax-1, step=1, continuous_update=False)
        )

semblance_numpy_file:/glb/data/cdis_projects/users/ussnis/attEngine/dataOut/fourpoint/fourpoint_semblance.npy
